# Phenotype classifcation using CellX 

This notebook shows how to take segmented time lapse microscopy images and use h2b fluorescence markers to classfiy mitotic state of the cell cycle. 

The sections of this notebook are as follows:

1. Load images
2. Localise the objects
3. Classify the objects
4. Batch process

The data used in this notebook is timelapse microscopy data with h2b-gfp/rfp markers that show the spatial extent of the nucleus and it's mitotic state. 

This notebook uses the dask octopuslite image loader from the CellX/Lowe lab project.

In [1]:
from octopuslite import DaskOctopusLiteLoader
import btrack
from tqdm.auto import tqdm
import numpy as np
from scipy.special import softmax
import os
import matplotlib.pyplot as plt
from skimage.io import imread, imshow
from cellx import load_model
from cellx.tools.image import InfinitePaddedImage
from skimage.transform import resize
import glob
%matplotlib inline
plt.rcParams['figure.figsize'] = [18,8]

## 1. Load segmentation images

#### *Important:* from this point on you will need to be consistent with the use of cropping and alignment. 
Using a previously generated alignment transformation will aid greatly in the tracking notebook, which depends on the object localisation performed in this notebook. Cropping your images will ensure that no border effects from the translational shift are seen. 

In [7]:
# load images
expt = 'ND0012'
pos = 'Pos5'
root_dir = '/home/nathan/data/kraken/ras'
image_path = f'{root_dir}/{expt}/{pos}/{pos}_images'
transform_path = f'{root_dir}/{expt}/{pos}/gfp_transform_tensor.npy'
images = DaskOctopusLiteLoader(image_path, 
                               transforms=transform_path,
                               crop=(1200,1600), 
                               remove_background=False)

Using cropping: (1200, 1600)


## 2. Localise the objects
We need to also measure the mean intensity regionprops parameter in order to differentiate object class, for which we need to provide an image to measure. This means we need to provide the segmentation images twice: once to find the centroid and once to measure the pixel intensity. 

In [7]:
objects = btrack.utils.segmentation_to_objects(
    images['mask'], images['mask'],
    properties = ('area', 'max_intensity', ),
)

[INFO][2022/02/04 10:18:09 AM] Localizing objects from segmentation...
[INFO][2022/02/04 10:18:09 AM] Found intensity_image data
[INFO][2022/02/04 10:18:09 AM] Calculating weighted centroids using intensity_image
[INFO][2022/02/04 10:28:10 AM] Objects are of type: <class 'dict'>
[INFO][2022/02/04 10:28:10 AM] ...Found 9566 objects in 1738 frames.


#### Can also assign measured values from raw image to each segment using `skimage.measure.regionprops` parameters
But also need to load the raw images to be measured first. Cannot currently save out `intensity_image` parameter to object file.

In [ ]:
detailed_objects = btrack.utils.segmentation_to_objects(
    images['mask'], 
    images['gfp'],
    properties = ('area', 'mean_intensity', 'intensity_image'), 
)

In [ ]:
detailed_objects[0]

Example image showing PCNA-iRFP morphology 

In [ ]:
imshow(detailed_objects[0].properties['intensity_image'])

## 2b. Differentiate the objects based on class ID

In [ ]:
objects_gfp = [obj for obj in objects if obj.properties['max_intensity'] == 1]
objects_rfp = [obj for obj in objects if obj.properties['max_intensity'] == 2]

## 3. Classify the objects 

Load model

In [2]:
model = load_model('../models/cellx_classifier_stardist.h5')

Define normalisation functions

In [3]:
def normalize_channels(x):

    for dim in range(x.shape[-1]):
        x[..., dim] = normalize(x[..., dim])
        
    return x

def normalize(x):

    xf = x.astype(np.float32)
    mx = np.mean(xf)
    sd = np.max([np.std(xf), 1./np.prod(x.shape)])

    return (xf - mx) / sd

Define classifier function

In [4]:
def classify_objects(image,  gfp, rfp, objects, obj_type):
    
    # define stages of cell cycle to classify (dependent on model type)
    LABELS = ["interphase", "prometaphase", "metaphase", "anaphase", "apoptosis"]
    
    # iterate over frames
    for n in tqdm(range(image.shape[0])):
        
        # only select objects if in frame
        _objects = [o for o in objects if o.t == n]
        
        # empty placeholder arrays
        crops = []
        to_update = []
        
        # select h2b channel to aid in classification
        fp = gfp if obj_type == 1 else rfp
        
        # create stack by computing each frame of dask array input
        frame = np.stack(
            [image[n, ...].compute(), fp[n, ...].compute()], 
            axis=-1,) 
        
        # create padded image for network
        vol = InfinitePaddedImage(frame, mode = 'reflect')
        
        # iterate over objects 
        for obj in _objects:
            
            # create coords for image slice
            xs = slice(int(obj.x-40), int(obj.x+40), 1)
            ys = slice(int(obj.y-40), int(obj.y+40), 1)
            
            # crop image
            crop = vol[ys, xs, :]
            crop = resize(crop, (64, 64), preserve_range=True).astype(np.float32)
            
            # normalise image
            if crop.shape == (64 ,64, 2):
                crops.append(normalize_channels(crop))
                to_update.append(obj)
            else:
                print(crop.shape)
                
        if not crops:
            continue
            
        # use classifcation model to predict
        pred = model.predict(np.stack(crops, axis=0))
        
        # check correct number of predictions
        assert pred.shape[0] == len(_objects)
        
        # assign labels to objects
        for idx in range(pred.shape[0]):
            obj = _objects[idx]
            
            # assigning details of prediction
            pred_label = np.argmax(pred[idx, ...])
            pred_softmax = softmax(pred[idx, ...])
            logits = {f"prob_{k}": pred_softmax[ki] for ki, k in enumerate(LABELS)}
            
            # write out
            obj.label = pred_label
            obj.properties = logits

    return objects

#### Load raw images for classifier, a colour channel dependent on `obj_type` needed too (i.e. GFP is `obj_type = 1`, RFP is `obj_type = 2`

In [ ]:
bf = images['brightfield']
gfp = images['gfp']
rfp = images['rfp']

#### Classify objects

In [ ]:
objects_gfp = classify_objects(bf, objects_gfp, obj_type = 1)
objects_rfp = classify_objects(bf, objects_rfp, obj_type = 2)

#### Inspect an example object

In [ ]:
objects_gfp[0]

#### Save out classified GFP objects

In [ ]:
with btrack.dataio.HDF5FileHandler(
    f'{root_dir}/{expt}/{pos}/objects_type_1.h5', 'w', obj_type='obj_type_1',
) as hdf:
    #hdf.write_segmentation(masks['mask'])
    hdf.write_objects(objects_gfp)

#### Save out classified RFP objects

In [ ]:
with btrack.dataio.HDF5FileHandler(
    f'{root_dir}/{expt}/{pos}/objects_type_2.h5', 'w', obj_type='obj_type_2',
) as hdf:
    #hdf.write_segmentation(masks['mask'])
    hdf.write_objects(objects_rfp)

# 4. Batch process
Iterate over many experiments and positions (need to ensure you define normalisation and classification functions above first)

In [ ]:
%%timeit
root_dir = '/home/nathan/data/kraken/ras'
expt_list = [#'ND0010', 'ND0011', 
    'ND0012', 
    #'ND0013'
]
#expt_list = [expt for expt in os.listdir(root_dir) if len(expt) == 6]

pos_list = 'all'
overwrite = False

for expt in tqdm(expt_list):
    try:
        # Find all positions in that experiment, if pos_list is all then it finds all positions
        if pos_list == 'all':
            pos_list = [pos for pos in os.listdir(f'{root_dir}/{expt}') 
                    if 'Pos' in pos 
                    and os.path.isdir(f'{root_dir}/{expt}/{pos}')]  

        ### Iterate over all positions in that experiment
        for pos in tqdm(natsorted(pos_list)):

            ### check if overwrite param is false check if raw directory already created and if type of transform file already exists and decide whether to skip pos
            if not overwrite and glob.glob(f'{root_dir}/{expt}/{pos}/*untrans_no_bg_objects_type*.h5'):
                print(glob.glob(f'{root_dir}/{expt}/{pos}/**untrans_no_bg_objects_type*.h5'), f'file found, skipping {expt}/{pos}')
                continue

            #check seg is complete
            mask_path = f'{root_dir}/{expt}/{pos}/{pos}_images/*channel099*'
            if glob.glob(mask_path) and len(glob.glob(mask_path)) == len(glob.glob(f'{root_dir}/{expt}/{pos}/{pos}_images/*channel001*')):
                print(expt, pos, 'enough masks')

                print(f'Starting {expt}/{pos}')
                # load segmentation images and apply necessary transforms and crops
                image_path = f'{root_dir}/{expt}/{pos}/{pos}_images'
                #transform_path = f'{root_dir}/{expt}/{pos}/gfp_transform_tensor.npy'
                images = DaskOctopusLiteLoader(image_path, 
                                   #transforms=transform_path,
                                   #crop=(1200,1600), 
                                   remove_background=False)

                # ID the objects in each segmentation image and assign option properties to them
                objects = btrack.utils.segmentation_to_objects(
                    images['mask'], 
                    properties = ('area',),
                    assign_class_ID = True
                )

                # differentiate the objects based on class ID
                objects_gfp = [obj for obj in objects if obj.properties['class id'] == 1]
                objects_rfp = [obj for obj in objects if obj.properties['class id'] == 2]

                # load classifcation model and define labels
                model = load_model('../models/cellx_classifier_stardist.h5')
                LABELS = ["interphase", "prometaphase", "metaphase", "anaphase", "apoptosis"]

                # load images for classifcation
                bf = images['brightfield']
                gfp = images['gfp']
                rfp = images['rfp']

                # classify objects
                print("Classifying objects")
                objects_gfp = classify_objects(bf, gfp, rfp, objects_gfp, obj_type = 1)
                objects_rfp = classify_objects(bf, gfp, rfp, objects_rfp, obj_type = 2)

                # save out classified objects as segmentation h5 file
                with btrack.dataio.HDF5FileHandler(
                    f'{root_dir}/{expt}/{pos}/untrans_objects_type_1.h5', 'w', obj_type='obj_type_1',
                ) as hdf:
                    #hdf.write_segmentation(masks['mask'])
                    hdf.write_objects(objects_gfp)
                with btrack.dataio.HDF5FileHandler(
                    f'{root_dir}/{expt}/{pos}/untrans_objects_type_2.h5', 'w', obj_type='obj_type_2',
                ) as hdf:
                    #hdf.write_segmentation(masks['mask'])
                    hdf.write_objects(objects_rfp)  
    except Exception as e: 
        print(e)
        #print(pos, expt, 'failed probably due to zero cell count')

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

['/home/nathan/data/kraken/ras/ND0012/Pos0/untrans_no_bg_objects_type_2.h5', '/home/nathan/data/kraken/ras/ND0012/Pos0/untrans_no_bg_objects_type_1.h5'] file found, skipping ND0012/Pos0
['/home/nathan/data/kraken/ras/ND0012/Pos1/untrans_no_bg_objects_type_2.h5', '/home/nathan/data/kraken/ras/ND0012/Pos1/untrans_no_bg_objects_type_1.h5'] file found, skipping ND0012/Pos1
ND0012 Pos2 enough masks
Starting ND0012/Pos2


[INFO][2022/02/23 12:22:18 PM] Localizing objects from segmentation...
[INFO][2022/02/23 12:26:20 PM] Objects are of type: <class 'dict'>
[INFO][2022/02/23 12:26:23 PM] ...Found 478183 objects in 1738 frames.


Classifying objects


  0%|          | 0/1738 [00:00<?, ?it/s]

# ND11 and ND12 processing

In [ ]:
%%timeit
root_dir = '/home/nathan/data/kraken/ras'
expt_list = [#'ND0010', 'ND0011', 
    'ND0011', 
    #'ND0013'
]
#expt_list = [expt for expt in os.listdir(root_dir) if len(expt) == 6]

pos_list = 'all'
overwrite = False

for expt in tqdm(expt_list):
    try:
        # Find all positions in that experiment, if pos_list is all then it finds all positions
        if pos_list == 'all':
            pos_list = [pos for pos in os.listdir(f'{root_dir}/{expt}') 
                    if 'Pos' in pos 
                    and os.path.isdir(f'{root_dir}/{expt}/{pos}')]  

        ### Iterate over all positions in that experiment
        for pos in tqdm(natsorted(pos_list)):

            ### check if overwrite param is false check if raw directory already created and if type of transform file already exists and decide whether to skip pos
            if not overwrite and glob.glob(f'{root_dir}/{expt}/{pos}/*untrans_no_bg_objects_type*.h5'):
                print(glob.glob(f'{root_dir}/{expt}/{pos}/**untrans_no_bg_objects_type*.h5'), f'file found, skipping {expt}/{pos}')
                continue

            #check seg is complete
            mask_path = f'{root_dir}/{expt}/{pos}/{pos}_images/*channel099*'
            if glob.glob(mask_path) and len(glob.glob(mask_path)) == len(glob.glob(f'{root_dir}/{expt}/{pos}/{pos}_images/*channel001*')):
                print(expt, pos, 'enough masks')

                print(f'Starting {expt}/{pos}')
                # load segmentation images and apply necessary transforms and crops
                image_path = f'{root_dir}/{expt}/{pos}/{pos}_images'
                #transform_path = f'{root_dir}/{expt}/{pos}/gfp_transform_tensor.npy'
                images = DaskOctopusLiteLoader(image_path, 
                                   #transforms=transform_path,
                                   #crop=(1200,1600), 
                                   remove_background=False)

                # ID the objects in each segmentation image and assign option properties to them
                objects = btrack.utils.segmentation_to_objects(
                    images['mask'], 
                    properties = ('area',),
                    assign_class_ID = True
                )

                # differentiate the objects based on class ID
                objects_gfp = [obj for obj in objects if obj.properties['class id'] == 1]
                objects_rfp = [obj for obj in objects if obj.properties['class id'] == 2]

                # load classifcation model and define labels
                model = load_model('../models/cellx_classifier_stardist.h5')
                LABELS = ["interphase", "prometaphase", "metaphase", "anaphase", "apoptosis"]

                # load images for classifcation
                bf = images['brightfield']
                gfp = images['gfp']
                rfp = images['rfp']

                # classify objects
                print("Classifying objects")
                objects_gfp = classify_objects(bf, gfp, rfp, objects_gfp, obj_type = 1)
                objects_rfp = classify_objects(bf, gfp, rfp, objects_rfp, obj_type = 2)

                # save out classified objects as segmentation h5 file
                with btrack.dataio.HDF5FileHandler(
                    f'{root_dir}/{expt}/{pos}/untrans_objects_type_1.h5', 'w', obj_type='obj_type_1',
                ) as hdf:
                    #hdf.write_segmentation(masks['mask'])
                    hdf.write_objects(objects_gfp)
                with btrack.dataio.HDF5FileHandler(
                    f'{root_dir}/{expt}/{pos}/untrans_objects_type_2.h5', 'w', obj_type='obj_type_2',
                ) as hdf:
                    #hdf.write_segmentation(masks['mask'])
                    hdf.write_objects(objects_rfp)  
    except Exception as e: 
        print(e)
        #print(pos, expt, 'failed probably due to zero cell count')

# copying all h5 files to temp directory to transfer home

In [2]:
root_dir = '/home/nathan/data/kraken/ras'


In [3]:
file_list = glob.glob( f'{root_dir}/*/*/untrans_no_bg_objects_type_1.h5')

In [4]:
from natsort import natsorted

In [5]:
natsorted(file_list)

['/home/nathan/data/kraken/ras/ND0010/Pos0/untrans_no_bg_objects_type_1.h5',
 '/home/nathan/data/kraken/ras/ND0010/Pos1/untrans_no_bg_objects_type_1.h5',
 '/home/nathan/data/kraken/ras/ND0010/Pos2/untrans_no_bg_objects_type_1.h5',
 '/home/nathan/data/kraken/ras/ND0010/Pos3/untrans_no_bg_objects_type_1.h5',
 '/home/nathan/data/kraken/ras/ND0010/Pos4/untrans_no_bg_objects_type_1.h5',
 '/home/nathan/data/kraken/ras/ND0010/Pos5/untrans_no_bg_objects_type_1.h5',
 '/home/nathan/data/kraken/ras/ND0010/Pos6/untrans_no_bg_objects_type_1.h5',
 '/home/nathan/data/kraken/ras/ND0010/Pos7/untrans_no_bg_objects_type_1.h5',
 '/home/nathan/data/kraken/ras/ND0010/Pos8/untrans_no_bg_objects_type_1.h5',
 '/home/nathan/data/kraken/ras/ND0010/Pos9/untrans_no_bg_objects_type_1.h5',
 '/home/nathan/data/kraken/ras/ND0010/Pos10/untrans_no_bg_objects_type_1.h5',
 '/home/nathan/data/kraken/ras/ND0010/Pos11/untrans_no_bg_objects_type_1.h5',
 '/home/nathan/data/kraken/ras/ND0012/Pos0/untrans_no_bg_objects_type_1.h5

In [6]:
import shutil, os

In [26]:
os.mkdir('/home/nathan/data/test/')

In [7]:
for file in tqdm(natsorted(file_list)):
    ##get path
    path = os.path.split(file)[0]
    filename = os.path.split(file)[-1]
    ## new path
    new_path = path.replace('/home/nathan/', '/home/nathan/temp2/')
    ## create new directory structure
    os.makedirs(new_path)
    new_file = os.path.join(new_path, filename)
    ## copy file to new location
    print(file)
    print(path)
    print(filename)
    print(new_path)
    print(new_file)
    shutil.copyfile(file, new_file)

  0%|          | 0/49 [00:00<?, ?it/s]

/home/nathan/data/kraken/ras/ND0010/Pos0/untrans_no_bg_objects_type_1.h5
/home/nathan/data/kraken/ras/ND0010/Pos0
untrans_no_bg_objects_type_1.h5
/home/nathan/temp2/data/kraken/ras/ND0010/Pos0
/home/nathan/temp2/data/kraken/ras/ND0010/Pos0/untrans_no_bg_objects_type_1.h5
/home/nathan/data/kraken/ras/ND0010/Pos1/untrans_no_bg_objects_type_1.h5
/home/nathan/data/kraken/ras/ND0010/Pos1
untrans_no_bg_objects_type_1.h5
/home/nathan/temp2/data/kraken/ras/ND0010/Pos1
/home/nathan/temp2/data/kraken/ras/ND0010/Pos1/untrans_no_bg_objects_type_1.h5
/home/nathan/data/kraken/ras/ND0010/Pos2/untrans_no_bg_objects_type_1.h5
/home/nathan/data/kraken/ras/ND0010/Pos2
untrans_no_bg_objects_type_1.h5
/home/nathan/temp2/data/kraken/ras/ND0010/Pos2
/home/nathan/temp2/data/kraken/ras/ND0010/Pos2/untrans_no_bg_objects_type_1.h5
/home/nathan/data/kraken/ras/ND0010/Pos3/untrans_no_bg_objects_type_1.h5
/home/nathan/data/kraken/ras/ND0010/Pos3
untrans_no_bg_objects_type_1.h5
/home/nathan/temp2/data/kraken/ras/ND0

/home/nathan/data/kraken/ras/ND0013/Pos11/untrans_no_bg_objects_type_1.h5
/home/nathan/data/kraken/ras/ND0013/Pos11
untrans_no_bg_objects_type_1.h5
/home/nathan/temp2/data/kraken/ras/ND0013/Pos11
/home/nathan/temp2/data/kraken/ras/ND0013/Pos11/untrans_no_bg_objects_type_1.h5
/home/nathan/data/kraken/ras/ND0013/Pos12/untrans_no_bg_objects_type_1.h5
/home/nathan/data/kraken/ras/ND0013/Pos12
untrans_no_bg_objects_type_1.h5
/home/nathan/temp2/data/kraken/ras/ND0013/Pos12
/home/nathan/temp2/data/kraken/ras/ND0013/Pos12/untrans_no_bg_objects_type_1.h5
/home/nathan/data/kraken/ras/ND0013/Pos13/untrans_no_bg_objects_type_1.h5
/home/nathan/data/kraken/ras/ND0013/Pos13
untrans_no_bg_objects_type_1.h5
/home/nathan/temp2/data/kraken/ras/ND0013/Pos13
/home/nathan/temp2/data/kraken/ras/ND0013/Pos13/untrans_no_bg_objects_type_1.h5
/home/nathan/data/kraken/ras/ND0013/Pos14/untrans_no_bg_objects_type_1.h5
/home/nathan/data/kraken/ras/ND0013/Pos14
untrans_no_bg_objects_type_1.h5
/home/nathan/temp2/data/

# need to do other set of files (with bg)

In [11]:
for file in file_list:
    

'/home/nathan/data/kraken/ras/*untrans_objects_type_2.h5'

# Parallel batch process

In [5]:
def classify(pos):
    ### check if overwrite param is false check if raw directory already created and if type of transform file already exists and decide whether to skip pos
    

    print(f'Starting {expt}/{pos}')
    # load segmentation images and apply necessary transforms and crops
    image_path = f'{root_dir}/{expt}/{pos}/{pos}_images'
    #transform_path = f'{root_dir}/{expt}/{pos}/gfp_transform_tensor.npy'
    images = DaskOctopusLiteLoader(image_path, 
                       #transforms=transform_path,
                       #crop=(1200,1600), 
                       remove_background=False)

    # ID the objects in each segmentation image and assign option properties to them
    objects = btrack.utils.segmentation_to_objects(
        images['mask'], images['mask'],
        properties = ('area', 'max_intensity', ),
    )

    # differentiate the objects based on class ID
    objects_gfp = [obj for obj in objects if obj.properties['max_intensity'] == 1]
    objects_rfp = [obj for obj in objects if obj.properties['max_intensity'] == 2]

    # load classifcation model and define labels
    model = load_model('../models/cellx_classifier_stardist.h5')
    LABELS = ["interphase", "prometaphase", "metaphase", "anaphase", "apoptosis"]

    # load images for classifcation
    bf = images['brightfield']
    gfp = images['gfp']
    rfp = images['rfp']

    # classify objects
    print("Classifying objects")
    objects_gfp = classify_objects(bf, objects_gfp, obj_type = 1)
    objects_rfp = classify_objects(bf, objects_rfp, obj_type = 2)

    # save out classified objects as segmentation h5 file
    with btrack.dataio.HDF5FileHandler(
        f'{root_dir}/{expt}/{pos}/objects_type_1_untrans.h5', 'w', obj_type='obj_type_1',
    ) as hdf:
        #hdf.write_segmentation(masks['mask'])
        hdf.write_objects(objects_gfp)
    with btrack.dataio.HDF5FileHandler(
        f'{root_dir}/{expt}/{pos}/objects_type_2_untrans.h5', 'w', obj_type='obj_type_2',
    ) as hdf:
        #hdf.write_segmentation(masks['mask'])
        hdf.write_objects(objects_rfp)     

    return

In [6]:
from multiprocessing import Pool
cpus = os.cpu_count()
cpus

12

In [11]:
pos_list = [pos for pos in os.listdir(f'{root_dir}/{expt}') 
                    if 'Pos' in pos 
                    and os.path.isdir(f'{root_dir}/{expt}/{pos}')]
pos_list

['Pos5',
 'Pos11',
 'Pos3',
 'Pos1',
 'Pos8',
 'Pos10',
 'Pos0',
 'Pos2',
 'Pos6',
 'Pos7',
 'Pos9',
 'Pos4']

In [10]:
root_dir = '/home/nathan/data/kraken/ras'
expt_list = [expt for expt in os.listdir(root_dir) if len(expt) == 6]

for expt in expt_list:
    pos_list = [pos for pos in os.listdir(f'{root_dir}/{expt}') 
                    if 'Pos' in pos 
                    and os.path.isdir(f'{root_dir}/{expt}/{pos}')]
    if __name__ == '__main__':
        with Pool(cpus) as p:
            p.map(classify, pos_list)

Starting ND0013/Pos13Starting ND0013/Pos11Starting ND0013/Pos8Starting ND0013/Pos1Starting ND0013/Pos10
Starting ND0013/Pos0
Starting ND0013/Pos2

Starting ND0013/Pos5
Starting ND0013/Pos3



Starting ND0013/Pos7
Starting ND0013/Pos6Starting ND0013/Pos14



[INFO][2022/02/12 02:53:37 PM] Localizing objects from segmentation...
[INFO][2022/02/12 02:53:37 PM] Found intensity_image data
[INFO][2022/02/12 02:53:37 PM] Calculating weighted centroids using intensity_image
[INFO][2022/02/12 02:53:38 PM] Objects are of type: <class 'dict'>
[INFO][2022/02/12 02:53:38 PM] ...Found 46 objects in 2 frames.
[INFO][2022/02/12 02:53:39 PM] Localizing objects from segmentation...
[INFO][2022/02/12 02:53:39 PM] Found intensity_image data
[INFO][2022/02/12 02:53:39 PM] Calculating weighted centroids using intensity_image
[INFO][2022/02/12 02:53:40 PM] Localizing objects from segmentation...
[INFO][2022/02/12 02:53:40 PM] Found intensity_image data
[INFO][2022/02/12 02:53:40 PM] Calculating weighted centroids using intensity_image
[INFO][2022/02/12 02:53:41 PM] Localizing objects from segmentation...
[INFO][2022/02/12 02:53:41 PM] Found intensity_image data
[INFO][2022/02/12 02:53:41 PM] Calculating weighted centroids using intensity_image
[INFO][2022/02/12

Starting ND0013/Pos9


[INFO][2022/02/12 02:53:42 PM] Localizing objects from segmentation...
[INFO][2022/02/12 02:53:42 PM] Found intensity_image data
[INFO][2022/02/12 02:53:42 PM] Calculating weighted centroids using intensity_image
[INFO][2022/02/12 02:53:43 PM] Localizing objects from segmentation...
[INFO][2022/02/12 02:53:43 PM] Found intensity_image data
[INFO][2022/02/12 02:53:43 PM] Calculating weighted centroids using intensity_image
[INFO][2022/02/12 02:53:43 PM] Localizing objects from segmentation...
[INFO][2022/02/12 02:53:43 PM] Found intensity_image data
[INFO][2022/02/12 02:53:43 PM] Calculating weighted centroids using intensity_image
[INFO][2022/02/12 02:53:56 PM] Localizing objects from segmentation...
[INFO][2022/02/12 02:53:56 PM] Found intensity_image data
[INFO][2022/02/12 02:53:56 PM] Calculating weighted centroids using intensity_image
Process ForkPoolWorker-10:


KeyboardInterrupt: 

Traceback (most recent call last):
  File "/home/nathan/analysis/miniconda3/envs/cellx/lib/python3.9/threading.py", line 312, in wait
    waiter.acquire()
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/nathan/analysis/miniconda3/envs/cellx/lib/python3.9/queue.py", line 171, in get
    self.not_empty.wait()
  File "/home/nathan/analysis/miniconda3/envs/cellx/lib/python3.9/threading.py", line 312, in wait
    waiter.acquire()
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/nathan/analysis/miniconda3/envs/cellx/lib/python3.9/multiprocessing/pool.py", line 125, in worker
    result = (True, func(*args, **kwds))
  File "/home/nathan/analysis/miniconda3/envs/cellx/lib/python3.9/multiprocessing/pool.py", line 48, in mapstar
    return list(map(*args))
  File "<ipython-input-5-b55b09ea950c>", line 15, in classify
    